In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import itertools
import os

### Scrape One Player

In [ ]:
def scrape_player(URL: str):
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = URL[-1]
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        # player_bio["games"] = game_l

        a = soup.find("table", class_="table").findAll("tr")
        a2 = a[1:]
        # print(a[1:])
        # print(len(a[1:]))
        list_a = [b.has_attr("class") for b in a[1:]]
        print(list_a)
        list_b = np.nonzero(list_a)[0]
        print(list(itertools.pairwise(list_b)))
        print(len(list(itertools.pairwise(list_b))) + 1)
        list_c = []
        for i in list(itertools.pairwise(list_b)):
            # print(a2[i[0]:i[1]])
            list_c.append(a2[i[0] : i[1]])
        # print(a2[list_b[-1]:])
        list_c.append(a2[list_b[-1] :])
        len(list_c)
        game_list = []
        for game in list_c:
            # print(game[0])
            # print([i.text.replace("\n", "") for i in game[0].findAll("td")[:3]])
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                )
            )
            # print(dict_cols)
            events = []
            for event in game[1:]:
                # print([j.text.replace("\n", "") for j in event.findAll("td")[1:-2]])
                dict_event = dict(
                    zip(
                        ["event", "team", "position", "medal"],
                        [j.text.replace("\n", "") for j in event.findAll("td")[1:-2]],
                    )
                )
                events.append(dict_event)
            # print(events)
            dict_cols["Events"] = events
            # print(dict_cols)
            game_list.append(dict_cols)
        # print(game_list)
        player_bio["games"] = game_list

    return player_bio

In [ ]:
scrape_player("https://www.olympedia.org/athletes/1")

In [ ]:
scrape_player("https://www.olympedia.org/athletes/93860")

### Scrape All Olympic Games Participants

In [ ]:
players = []
for i in range(1, 101):
    URL = f"https://www.olympedia.org/athletes/{i}"
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = i
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        player_bio["games"] = game_l

        players.append(player_bio)

with open("players.json", "w") as f:
    json.dump(players, f)

Problem: There are about 150 000 players to scrape all this data we need to split it into threads but also send these threads to different computers

### Scrape only players who won medals

We need selenium to disable the limit of displayed players and to get all medal havers

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
def get_medal_havers():
    URL = "https://www.olympedia.org/statistics/medal/athlete"
    page2 = requests.get(URL)

    soup2 = BeautifulSoup(page2.content, "html.parser")
    table = soup2.find("table", class_="table")

    col_names = [i.text for i in table.find("thead").findAll("th")]
    # print(col_names)

    # create webdriver object
    driver = webdriver.Firefox()
    driver.get("https://www.olympedia.org/statistics/medal/athlete")

    x = driver.find_element(By.NAME, "limit")
    drop = Select(x)

    # select by visible text
    drop.select_by_visible_text("Show all")
    # wait for all the players to be displayed on the site
    time.sleep(20)

    players = driver.find_elements(By.CLASS_NAME, "top")
    out = [i.get_attribute("innerHTML") for i in players]

    driver.close()

    players_medal = []
    for line in out:
        soup = BeautifulSoup(line, "html.parser")
        players_medal.append([i.text.lstrip() for i in soup.findAll("td")])

    df = pd.DataFrame(players_medal, columns=col_names)
    df.to_csv("medal_havers.csv", index=False)

    return df

In [ ]:
get_medal_havers()

## Try Threading to get all the players

In [3]:
import threading
import logging

In [ ]:
def scrape_players(r: int):
    logging.info(f"Starting Thread in range {r} - {r + 100}")
    players = []
    for i in range(r, r + 100):
        URL = f"https://www.olympedia.org/athletes/{str(i)}"
        # print(URL)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find("div", class_="athlete_bio")

        if results != None:
            table = results.find("table", class_="biodata")
            player_bio = {}
            player_bio["id"] = i
            if results.find("img", class_="photo"):
                player_bio["photo_link"] = results.find("img", class_="photo")["src"]
            for player in table.findAll("tr"):
                player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                    "td"
                ).text.lstrip()

            games = soup.find("table", class_="table").findAll("tr")
            a2 = games[1:]
            list_a = [b.has_attr("class") for b in a2]
            list_b = np.nonzero(list_a)[0]

            list_c = []
            for i in list(itertools.pairwise(list_b)):
                list_c.append(a2[i[0] : i[1]])
            list_c.append(a2[list_b[-1] :])

            game_list = []
            for game in list_c:
                dict_cols = dict(
                    zip(
                        ["game", "discipline", "noc"],
                        [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                    )
                )
                events = []
                for event in game[1:]:
                    dict_event = dict(
                        zip(
                            ["event", "team", "position", "medal"],
                            [
                                j.text.replace("\n", "")
                                for j in event.findAll("td")[1:-2]
                            ],
                        )
                    )
                    events.append(dict_event)
                dict_cols["Events"] = events
                game_list.append(dict_cols)
            player_bio["games"] = game_list

            print(player_bio)

            players.append(player_bio)
    with open(f"players/players_{r}.json", "w") as f:
        json.dump(players, f)
    logging.info(f"Thread finished {r} - {r + 100}")
    return players

Inside each batch there is 1000 players and there are 150 batches, which is a lot and must be optimized

In [ ]:
def thread_func(batch):
    thread_list = []
    for i in batch:
        thread = threading.Thread(target=scrape_players, args=(i,))
        thread_list.append(thread)
        thread.start()
    for t in thread_list:
        t.join()

In [ ]:
batches = []
for i in list(range(1, 150000, 1000)):
    batches.append([i] + [y * 100 + i for y in range(1, 10)])

In [ ]:
print(batches[1:])

In [ ]:
# for batch in batches:
#     thread_func(batch)

### Create CSV file 

In [ ]:
sorted_files = ["players_" + str(y) + ".json" for y in sorted([int(x.replace("players_", "").replace(".json", "")) for x in os.listdir("players/")])]

In [ ]:
sorted_files

In [ ]:
sorted_files[:2]

In [ ]:
python_objects = []

# Load each JSON file into a Python object.
for json_file in sorted_files:
    with open(f"players/{json_file}", "r") as f:
        for player in json.load(f):
            python_objects.append(player)

In [ ]:
len(python_objects)

In [ ]:
with open(f"all_players.json", "w") as f:
    json.dump(python_objects, f)

In [ ]:
with open("all_players.json", 'r') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df["games"]

In [ ]:
data2 = pd.json_normalize(data, record_path=["games"], meta=["id"])

In [ ]:
data2.Events

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data[0])

temp = pd.json_normalize(data[0], record_path=["games"])
# temp2 = pd.json_normalize(temp, record_path=["Events"])

In [ ]:
Final = pd.concat([df.drop(['games'], axis = 1), temp], axis = 1)
Final

In [ ]:
data[0]

In [ ]:
len(data[1:])

In [ ]:
df = pd.DataFrame(data[0])
temp = pd.json_normalize(data[0], record_path=["games"])
Final = pd.concat([df.drop(['games'], axis = 1), temp], axis = 1)
for player in data[1:]:
    df2 = pd.DataFrame(player)
    temp2 = pd.json_normalize(player, record_path=["games"])
    new_df = pd.concat([df2.drop(['games'], axis = 1), temp2], axis = 1)
    Final = pd.concat([Final, new_df], ignore_index=True, sort=False)
    print(len(Final))

In [ ]:
Final

In [ ]:
Final.to_json(orient="records")

In [ ]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)

In [ ]:
pd.json_normalize(Final.Events[1])

In [4]:
with open("final.json", 'r') as f:
    data = json.load(f)

In [7]:
len(data)/2

104060.0

In [12]:
len(data[:104060])

104060

In [15]:
len(data[104060:])

104060

In [ ]:
with open("final_part1.json", "w") as f:
    json.dump(data[:104060], f)
with open("final_part2.json", "w") as f:
    json.dump(data[104060:], f)